## 변수 범위
*해 아래 새로운 것은 없습니다.*

유효범위는 함수입니다. 함수가 끝나면 변수도 사라져요.

In [2]:
def f1():
    v1 = 10
f1()
v1

NameError: name 'v1' is not defined

**지역변수 -> 전역변수** 순서로 변수를 찾아갑니다. (정확히는 LEGB rule. Local Enclosing Global Builtin순서로 찾습니다.)

In [1]:
# shadowing
a = 10
def f1():
    a = 20
    print(a)
print(a)
f1()
print(a)

10
20
10


In [3]:
# 왜 에러가 나지 않을까요?
def f1(a):
    print(a)
    print(b)

In [4]:
# 실행시점에 에러가 납니다.
f1(11)

11


NameError: name 'b' is not defined

In [5]:
# 실행시점에 b를 찾겠네요.
b = 10
f1(11)

11
10


In [6]:
# 이상한 문제
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(10)

10


UnboundLocalError: local variable 'b' referenced before assignment

- python은 line단위로 실행하는 것으로 알고 있는데?
- **실제로는 함수의 body를 통채로 compile합니다.** compile후에 한 라인씩 수행.
- 컴파일 결과 b는 local변수로 결정되었으며 print(b) 수행 시점에 local에서 b를 찾습니다.

In [7]:
b = 6
def f2(a):
    global b  # 이게 필요합니다.
    print(a)
    print(b)
    b = 9

전역변수를 참조하는 것은 상관없지만 값을 할당하려고 한다면 global을 챙겨주세요.

In [10]:
x = 10
def foo():
    x += 1
    print(x)
foo()

UnboundLocalError: local variable 'x' referenced before assignment

In [9]:
lst = [1,2,3]
def foo2():
    lst += [5]
foo2()

UnboundLocalError: local variable 'lst' referenced before assignment

In [22]:
# 잠깐 복습, 다음의 차이는?
lst = [1,2,3]
lst.extend([4,5])
print(lst)

lst2 = [1,2,3]
lst2 = lst2 + [4,5]
print(lst2)

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


![](img/lst-visualize.png)

In [24]:
lst = [1,2,3]
def foo2():
    #lst += [5]
    lst.extend([5])
foo2()
lst

[1, 2, 3, 5]

(참고) 내가 원하는 context에서 local, global 변수를 찾아볼 수 있습니다.

In [33]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b = 9
    print(locals())
    print(globals()['b'])
f2(1)
print(globals()['b'])

1
6
{'a': 1}
9
9


## CLOSURE

함수내 함수를 정의하는 것을 단순히 closure라고 하지는 않습니다. 그렇지만 함수내 함수를 쓰면서만 closure가 발생하는 것은 맞습니다.

**확장된 변수 범위**를 갖는 **함수**를 closure라고 합니다.
- 함수 내에서 참조할 수 있지만 함수 내 선언되어 있지는 않습니다.
- 함수몸체 밖에 있는 non-global변수를 참조할 수 있습니다.
- 이 변수들을 free-variable이라고 합니다. (*global이 아니면서 local scope에 bound되지 않는 변수*)

In [32]:
# 아직 class는 모르실 수도 있지만;
class Averager():
    def __init__(self):
        self.series = []
    def calculate(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
avg.calculate(10)

10.0

In [30]:
# 함수로 간단히(직관적으로) 표현하고자 할 때
def make_averager():
    series = []  # free-variable이라고 합니다.
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

avg = make_averager()
avg(10)

10.0

- encapsulated된 변수범위를 포함한 함수를 표현할 때 사용합니다.
- closure 함수는 물론 class로도 표현할 수 있어요.  private 영역을 포함한 객체인스턴스를 생성하는 것과 유사하다고 볼 수 있어요.
- 함수로 표현하는게 간단할때가 있고, class에서의 self 표기법을 탈출할 수 있습니다.

함수의 free-variable을 검사할 수 있어요.

> In computer programming, the term free variable refers to variables used in a function that are neither local variables nor parameters of that function. The term non-local variable is often a synonym in this context.

https://en.wikipedia.org/wiki/Free_variables_and_bound_variables


In [31]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)
print(avg.__closure__[0].cell_contents)

('new_value', 'total')
('series',)
[10]


In [27]:
# inner는 closure일까요?

a = 1
def outer(b):
    c = 2
    def inner(d):
        e = 10
        return d*e
    return inner

oh = outer(10)
oh(30)

300

In [28]:
print(oh.__code__.co_varnames)
print(oh.__code__.co_freevars)
print(oh.__closure__)

('d', 'e')
()
None


In [8]:
# 나는 어느 c를 보아야만 하는가
a = 1
c = 1
def outer(b):
    c = 2
    def inner(d):
        global c  # or nonlocal c로 테스트해보세요
        c = 30
        e = c*10
        return d*e
    return inner

oh = outer(10)
oh(30)

9000

In [10]:
# closure가 사용될때 나오는 흔한 예
def create_multipliers():
    return [lambda x: i * x for i in range(5)]
for multiplier in create_multipliers():
    print(multiplier(2))

8
8
8
8
8


In [12]:
# closure가 사용될때 나오는 흔한 예(수정됨)
def create_multipliers():
    return [lambda x, i=i: i * x for i in range(5)]
for multiplier in create_multipliers():
    print(multiplier(2))

0
2
4
6
8


문제를 발견하고 이해할 수 있을 정도로만 파악합시다. 저는 Python에서는 위 문제를 경험해보진 않았어요.

## NONLOCAL

- nonlocal == global?
- nonlocal == free-var!
- python3에서 추가된 keyword. free-variable로 유지시켜줌.

In [34]:
b = 6
def f2(a):
    nonlocal b
    b = a
    print(b)
f2(3)
print(b)

SyntaxError: no binding for nonlocal 'b' found (<ipython-input-34-4d50293a816e>, line 3)

In [35]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [36]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        global count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)

NameError: name 'count' is not defined

In [37]:
count = 0
total = 0
def make_averager():
    def averager(new_value):
        global count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)
avg(20)
count = 10
avg(30)

5.454545454545454

In [39]:
# 우리가 원하는 결과
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)
avg(20)
count = 10
avg(30)

20.0

당장 nonlocal을 어떻게 써야 하지 걱정하지 않아도 됩니다. 다른 사람의 코드를 읽을때 당황하지 않는 것부터 시작합니다.

좀 더 이해하기는 쉬었던 global자체도 유익한 것은 않습니다. 전역변수는 오랜 프로그래밍 역사동안 권장되어 오지 않았습니다.

EXERCISE

In [ ]:
# 다음 실행결과를 예측해봅시다.
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)